In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import spacy
import matplotlib.pyplot as plt
import time
import multiprocessing
import zipfile
from spacy.tokenizer import Tokenizer
from shutil import rmtree
import time


base_loc = "/data/"
raw_zip_file = "s3_data/kaggle_coleridge/raw/coleridgeinitiative-show-us-the-data.zip"
unzip_loc = "s3_data/kaggle_coleridge/processed/unzipped/"

test_folder = "s3_data/kaggle_coleridge/processed/unzipped/test/"
train_folder = "s3_data/kaggle_coleridge/processed/unzipped/train/"

train_y_file = "train.csv"
submission_file = "sample_submission.csv"

output_folder = "s3_data/kaggle_coleridge/processed/processing_1/"

nlp = spacy.load("en_core_web_sm")

In [2]:
if not os.path.exists(os.path.join(base_loc, unzip_loc)):
    os.makedirs(os.path.join(base_loc, unzip_loc))
else:
    rmtree(os.path.join(base_loc, unzip_loc))
    os.makedirs(os.path.join(base_loc, unzip_loc))
    

with zipfile.ZipFile(os.path.join(base_loc, raw_zip_file), 'r') as zip_ref:
    zip_ref.extractall(os.path.join(base_loc, unzip_loc))

In [4]:
# def jaccard(str1, str2): 
#     a = set(str1.lower().split()) 
#     b = set(str2.lower().split())
#     c = a.intersection(b)
#     return float(len(c)) / (len(a) + len(b) - len(c))


# def clean_text(txt):
#     return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [6]:
def find_label_in_text(sent_tokens, label_tokens):
    max_loc = len(sent_tokens) - len(label_tokens)
    for i in range(0, max_loc):
        temp_tokens = sent_tokens[i:i+len(label_tokens)]
        
        if temp_tokens == label_tokens:
            return [i,  i-1+len(label_tokens)]
    return []


def get_tokens_loc(sent, sent_tokens):
    tokens_loc_start, tokens_loc_end = [], []
    
    start_search = 0
    for token in sent_tokens:
        temp_loc = sent.find(token, start_search)
        
        assert temp_loc != -1, "token <%s> not found in sentence <%s>"%(token, sent)
        tokens_loc_start.append(temp_loc)
        tokens_loc_end.append(temp_loc + len(token))
        
        start_search += len(token)
        
    return tokens_loc_start, tokens_loc_end
    
    
def get_sentences(ind, base_loc, train_folder, file_id, pub_title,dataset_title, dataset_label):

    
#     print("processing %s started"%ind)
    temp_file_loc = os.path.join(base_loc, train_folder, file_id+".json")
    json_data = json.load(open(temp_file_loc, "r"))
    sentences = []
    
    label_doc = nlp(dataset_label)
    label_tokens = [str(token) for token in label_doc]
    
    for elem in json_data:
        temp_text = elem["text"]
        temp_doc = nlp(temp_text)
        sents = list(temp_doc.sents)
        for s_ind, sent in enumerate(sents):
            sent = str(sent)
            
            sent_doc = nlp(sent)
            sent_tokens = [str(token) for token in sent_doc]
            sent_tokens_start, sent_tokens_end = get_tokens_loc(sent, sent_tokens)
            
            locations = find_label_in_text(sent_tokens, label_tokens)
            
            if locations:
                sentences.append({
                    "file": file_id+ ".json",
                    "publication": pub_title,
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "tokens": sent_tokens,
                    "dataset_title": dataset_title,
                    "label": dataset_label,
                    "label_start_tokens": locations[0],
                    "bael_end_tokens": locations[1],
                    "label_start": sent_tokens_start[locations[0]],
                    "label_end": sent_tokens_end[locations[1]]
                    
                })
            else:
                sentences.append({
                    "file": file_id+ ".json",
                    "publication": pub_title,
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "tokens": sent_tokens,
                })
#     print("Processing %s finished"%ind)
    return sentences
def get_sentences_process(ind, base_loc, train_folder, file_id, pub_title,dataset_title, dataset_label, q):

    
#     print("processing %s started"%ind)
    temp_file_loc = os.path.join(base_loc, train_folder, file_id+".json")
    json_data = json.load(open(temp_file_loc, "r"))
    sentences = []
    
    label_doc = nlp(dataset_label)
    label_tokens = [str(token) for token in label_doc]
    
    for elem in json_data:
        temp_text = elem["text"]
        temp_doc = nlp(temp_text)
        sents = list(temp_doc.sents)
        for s_ind, sent in enumerate(sents):
            sent = str(sent)
            
            sent_doc = nlp(sent)
            sent_tokens = [str(token) for token in sent_doc]
            sent_tokens_start, sent_tokens_end = get_tokens_loc(sent, sent_tokens)
            
            locations = find_label_in_text(sent_tokens, label_tokens)
            
            if locations:
                sentences.append({
                    "file": file_id+ ".json",
                    "publication": pub_title,
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "tokens": sent_tokens,
                    "dataset_title": dataset_title,
                    "label": dataset_label,
                    "label_start_tokens": locations[0],
                    "bael_end_tokens": locations[1],
                    "label_start": sent_tokens_start[locations[0]],
                    "label_end": sent_tokens_end[locations[1]]
                    
                })
            else:
                sentences.append({
                    "file": file_id+ ".json",
                    "publication": pub_title,
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "tokens": sent_tokens,
                })
#     print("Processing %s finished"%ind)
    return q.put(sentences)

In [7]:
train_df = pd.read_csv(os.path.join(base_loc, unzip_loc,  train_y_file))
inputs = []
count = 0
for i in range(train_df.shape[0]):
    inputs.append((
        count,
        base_loc,
        train_folder,
        train_df.loc[i, "Id"],
        train_df.loc[i, "pub_title"],
        train_df.loc[i, "dataset_title"],
        train_df.loc[i, "dataset_label"]
    ))
    count += 1

print(len(inputs))
print(inputs[0])

19661
(0, '/data/', 's3_data/kaggle_coleridge/processed/unzipped/train/', 'd0fa7568-7d8e-4db9-870f-f9c6f668c17b', 'The Impact of Dual Enrollment on College Degree Attainment: Do Low-SES Students Benefit?', 'National Education Longitudinal Study', 'National Education Longitudinal Study')


In [8]:
inputs= inputs[:10]

print("processing started")
st = round(time.time(), 0)

# ORDINARY LOOP
# output = [get_sentences(*i) for i in inputs]


# POOL
# process_pool = multiprocessing.Pool(3)
# output = process_pool.starmap(get_sentences, inputs)


# # PROCESS
pools = []
q = multiprocessing.Queue()
print(1)
for inp in inputs:
    temp_inp = (inp[0], inp[1], inp[2], inp[3], inp[4], inp[5], inp[6], q)
    p = multiprocessing.Process(target=get_sentences_process, args=temp_inp)
    
    p.start()
    pools.append(p)

output = []
print(2, len(pools))
for p_ind, p in enumerate(pools):
    output.append(q.get())
    print("closing", p_ind)
    p.join()
    print("closed", p_ind)


en = round(time.time(), 0)
print("program completed in %s seconds"%(en-st))

print(len(output))
print(output[0])

processing started
1
2 10
closing 0


Process Process-15:
Process Process-17:
Process Process-12:
Process Process-11:
Process Process-13:
Process Process-19:
Traceback (most recent call last):
Process Process-18:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    u

KeyboardInterrupt: 

KeyboardInterrupt
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
KeyboardInterrupt


In [13]:
print(len(output))
print(sum([len(o) for o in output]))

100
22853


In [5]:
print(len(output))
positive_sentences = [o for o in output if "label" in o]
negative_sentences = [o for o in output if "label" not in o]
print(len(positive_sentences))
print(len(negative_sentences))

2461475
37667
2423808


In [6]:
positive_output_file = os.path.join(base_loc, output_folder, "pos.json")
negative_output_file = os.path.join(base_loc, output_folder, "neg.json")

json.dump(positive_sentences, open(positive_output_file, 'w'))
json.dump(negative_sentences, open(negative_output_file, 'w'))
